<a href="https://colab.research.google.com/github/1kaiser/test2023/blob/main/OpenAltimetry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

OpenAltimetry ICESat-2: Elevation Workflow
Overview


This notebook pulls data from ICESat-2 surface-specific products - ATL06, ATL07, ATL08, ATL10, ATL12, ATL13 via the OpenAltimetry API.


Instructions
Replace the "OA_API_URL" string with the data copied from : "Copy API URL" from the OpenAltimetry elevation chart and start visualizing your data!
Author: Minh Phan - mnphan@ucsd.edu. Based off the work of Luis Lopez - luis.lopez@nsidc.org

In [62]:
import json
import requests

# Bounding box coordinates
left   = 75.979728
right  = 77.866667
bottom = 31.453599
top    = 32.416667

date = '2019-10-26'
trackid = 447


OA_API_URL = 'https://openaltimetry.org/data/api/icesat2/atl06?date='+date+'&minx='+str(left)+'&miny='+str(bottom)+'&maxx='+str(right)+'&maxy='+str(top)+'&trackId='+str(trackid)+'&beamName=gt3r&beamName=gt3l&beamName=gt2r&beamName=gt2l&beamName=gt1r&beamName=gt1l&outputFormat=json'

OA_API_URL += '&client=jupyter';

print(OA_API_URL)

r = requests.get(OA_API_URL)
data = r.json()


https://openaltimetry.org/data/api/icesat2/atl06?date=2019-10-26&minx=75.979728&miny=31.453599&maxx=77.866667&maxy=32.416667&trackId=447&beamName=gt3r&beamName=gt3l&beamName=gt2r&beamName=gt2l&beamName=gt1r&beamName=gt1l&outputFormat=json&client=jupyter


In [63]:
data

{'product': 'ATL06',
 'scForward': True,
 'trackId': 447,
 'series': [{'beam': 'gt3r',
   'isStrongBeam': True,
   'lat_lon_elev': [[32.41654703415639, 77.34211547864146, 5180.824],
    [32.416366097818205, 77.34210633966643, 5124.139],
    [32.41618547908113, 77.34209337539642, 5086.2085],
    [32.41600561302678, 77.342071334818, 5090.022],
    [32.415825657540466, 77.3420503685615, 5090.3003],
    [32.41564565191409, 77.34203001178935, 5089.8193],
    [32.41546574195223, 77.34200849739938, 5097.7847],
    [32.41528551975446, 77.34199076208921, 5088.779],
    [32.41510489258915, 77.34197788636538, 5058.964],
    [32.41492440639438, 77.34196332892714, 5036.4614],
    [32.41474421384178, 77.34194522478685, 5029.9673],
    [32.41456416157509, 77.34192542654966, 5030.142],
    [32.41438421294989, 77.34190438159294, 5035.1753],
    [32.41420432723775, 77.34188257550113, 5041.6807],
    [32.414024444373716, 77.34186073569396, 5046.7427],
    [32.413844559438004, 77.34183890571256, 5050.472]

In [68]:
import pandas as pd
import json
import requests
import plotly.graph_objects as go
from plotly.offline import iplot


# Bounding box coordinates
left   = 75.979728
right  = 77.866667
bottom = 31.453599
top    = 32.416667

date = '2019-10-26'
trackid = 447

# We populate a list with the elevation data using the OpenAltimetry API
OA_API_URL = 'https://openaltimetry.org/data/api/icesat2/atl06?date='+date+'&minx='+str(left)+'&miny='+str(bottom)+'&maxx='+str(right)+'&maxy='+str(top)+'&trackId='+str(trackid)+'&beamName=gt3r&beamName=gt3l&beamName=gt2r&beamName=gt2l&beamName=gt1r&beamName=gt1l&outputFormat=json'

OA_API_URL += '&client=jupyter';

OA_BEAMS = ['gt1r', 'gt1l', 'gt2r', 'gt2l', 'gt3r', 'gt3l']

df_beams = []
oa_plots = []

# This function will request the 6 tracks using OpenAltimetry's API
def getData(url):
    series = []
    b_url = url

    print('Requesting data from OA')
    r = requests.get(b_url)
    data = r.json()
    return data

date = '2019-10-26'
trackid = 447
# We populate a list with the elevation data using the OpenAltimetry API
OA_API_URL1 = 'https://openaltimetry.org/data/api/icesat2/atl06?date='+date+'&minx='+str(left)+'&miny='+str(bottom)+'&maxx='+str(right)+'&maxy='+str(top)+'&trackId='+str(trackid)+'&beamName=gt3r&beamName=gt3l&beamName=gt2r&beamName=gt2l&beamName=gt1r&beamName=gt1l&outputFormat=json'


date = '2022-07-30'
trackid = 592
# We populate a list with the elevation data using the OpenAltimetry API
OA_API_URL2 = 'https://openaltimetry.org/data/api/icesat2/atl06?date='+date+'&minx='+str(left)+'&miny='+str(bottom)+'&maxx='+str(right)+'&maxy='+str(top)+'&trackId='+str(trackid)+'&beamName=gt3r&beamName=gt3l&beamName=gt2r&beamName=gt2l&beamName=gt1r&beamName=gt1l&outputFormat=json'

a =[]
a.append(getData(OA_API_URL1))
a.append(getData(OA_API_URL2))

for elevation_data in a:
  product = elevation_data["product"]

  print('Reading data product: ' + product)

  for series_beam in elevation_data['series']:
      if any(word in series_beam['beam'] for word in OA_BEAMS):
          series = []
          
          data_name = 'lat_lon_elev'
          if product == 'ATL08':
              data_name = 'lat_lon_elev_canopy'

          for p in series_beam[data_name]:
              series.append({
                  'lat': p[0],
                  'lon': p[1],
                  'h': p[2]
              })
              
          print ('beam ' + series_beam['beam'] + ' - len = ' + str(len(series)))

          if (len(series) > 0):
              df = pd.DataFrame.from_dict(series)
              df.name = series_beam['beam']
              df_beams.append(df)
              
  print('df_beams len: ' + str(len(df_beams)))

for df in df_beams:
    oa_plots.append(go.Scatter3d(name=df.name, x=df['lat'], y=df['lon'], z=df['h'],
                                    marker=dict(
                                        size=2,
                                        color=df['h'],
                                        colorscale='Viridis',   # choose a colorscale
                                        opacity=0.8
                                    ),
                                    line=dict(
                                        color='darkblue',
                                        width=2
                                    )
                                )
                   )

layout = go.Layout(
    width=1200,
    height=600,
    scene = dict(aspectmode = 'manual', aspectratio =dict(x=1, y=1, z=0.5),
                 xaxis=dict(title='Latitude'), yaxis=dict(title='Longitude'), zaxis=dict(title='Elevation (m)'))
)

print('Plotting...')

fig = go.Figure(data=oa_plots, layout=layout)

iplot(fig)


Requesting data from OA
Requesting data from OA
Reading data product: ATL06
beam gt3r - len = 3831
beam gt3l - len = 3283
beam gt2r - len = 3753
beam gt2l - len = 3204
beam gt1r - len = 3891
beam gt1l - len = 3403
df_beams len: 6
Reading data product: ATL06
beam gt3r - len = 1591
beam gt3l - len = 782
beam gt2r - len = 1832
beam gt2l - len = 889
beam gt1r - len = 2199
beam gt1l - len = 857
df_beams len: 12
Plotting...


https://openaltimetry.org

In [58]:
a = !curl -X GET "https://openaltimetry.org/data/api/icesat2/atl06?date=2019-10-26&minx=75.979728&miny=31.453599&maxx=77.866667&maxy=32.416667&trackId=447&client=portal&outputFormat=json" -H "accept: */*"

In [59]:
a

['{"product": "ATL06", "scForward": true, "trackId": 447, "series": [{"beam": "gt1l","isStrongBeam": false,"lat_lon_elev": [[32.415826947054,77.40977539732262,5759.157],[32.41564708677463,77.40975318307426,5779.3994],[32.41546696842656,77.40973402327279,5790.8433],[32.41528629887673,77.40972147550099,5785.549],[32.415105393849394,77.40971175233625,5772.1113],[32.41492495041405,77.40969648237069,5773.182],[32.41474467385934,77.40967925787311,5780.378],[32.41456423579399,77.40966391104683,5781.9365],[32.41438357378778,77.4096512605006,5775.57],[32.414202737380144,77.40964071701455,5764.6562],[32.414021949992026,77.40962957197803,5755.673],[32.413841277340985,77.40961705478861,5748.877],[32.41366069445406,77.40960347947998,5745.9023],[32.41348014584254,77.4095895156828,5743.6367],[32.41329949781497,77.40957661645751,5739.281],[32.413118978922355,77.40956228691743,5738.135],[32.412938519833546,77.40954722162655,5739.3096],[32.41275794879942,77.40953351183657,5736.871],[32.41257727149889,77

In [67]:
!wget https://openaltimetry.org/data/api/icesat2/getTracks?date=2019-10-26&minx=75.979728&miny=31.453599&maxx=77.866667&maxy=32.416667&outputFormat=json -O a.json


/bin/bash: -O: command not found
--2023-01-30 22:10:46--  https://openaltimetry.org/data/api/icesat2/getTracks?date=2019-10-26
Resolving openaltimetry.org (openaltimetry.org)... 169.228.63.181
Connecting to openaltimetry.org (openaltimetry.org)|169.228.63.181|:443... connected.
HTTP request sent, awaiting response... 200 
Length: 212 [application/json]
Saving to: ‘getTracks?date=2019-10-26.1’

getTracks?date=2019 100%[===================>]     212  --.-KB/s    in 0s      

2023-01-30 22:10:46 (27.5 MB/s) - ‘getTracks?date=2019-10-26.1’ saved [212/212]

